In [219]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [220]:
df = pd.read_csv('dados_avc.csv')
df.head(10)


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
5,56669,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
6,53882,Male,74.0,1,1,Yes,Private,Rural,70.09,27.4,never smoked,1
7,10434,Female,69.0,0,0,No,Private,Urban,94.39,22.8,never smoked,1
8,27419,Female,59.0,0,0,Yes,Private,Rural,76.15,NaN,Unknown,1
9,60491,Female,78.0,0,0,Yes,Private,Urban,58.57,24.2,Unknown,1


In [221]:
# Preencher valores ausentes em 'bmi' com a média
df['bmi'].replace('N/A', float('nan'), inplace=True)
df['bmi'].fillna(df['bmi'].mean(), inplace=True)

# Converter colunas categóricas usando one-hot encoding
df = pd.get_dummies(df, columns=['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status'], drop_first=True)
df.head(20)


,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Male,gender_Other,ever_married_Yes,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,9046,67.0,0,1,228.69,36.600000,1,True,False,True,False,True,False,False,True,True,False,False
1,51676,61.0,0,0,202.21,28.893237,1,False,False,True,False,False,True,False,False,False,True,False
2,31112,80.0,0,1,105.92,32.500000,1,True,False,True,False,True,False,False,False,False,True,False
3,60182,49.0,0,0,171.23,34.400000,1,False,False,True,False,True,False,False,True,False,False,True
4,1665,79.0,1,0,174.12,24.000000,1,False,False,True,False,False,True,False,False,False,True,False
5,56669,81.0,0,0,186.21,29.000000,1,True,False,True,False,True,False,False,True,True,False,False
6,53882,74.0,1,1,70.09,27.400000,1,True,False,True,False,True,False,False,False,False,True,False
7,10434,69.0,0,0,94.39,22.800000,1,False,False,False,False,True,False,False,True,False,True,False
8,27419,59.0,0,0,76.15,28.893237,1,False,False,True,False,True,False,False,False,False,False,False
9,60491,78.0,0,0,58.57,24.200000,1,False,False,True,False,True,False,False,True,False,False,False


In [222]:
X = df.drop(['stroke', 'id','age', 'bmi'], axis=1)
y = df['stroke']


In [223]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [224]:
model = LogisticRegression(max_iter=1000)  # Aumentar max_iter se necessário
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [225]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy * 100:.2f}%")


Acurácia do modelo: 94.19%


In [226]:
# Pegar os coeficientes (pesos) do modelo treinado
coeficientes = model.coef_[0]

print(model.classes_)
# Criar um DataFrame com os nomes das características e seus coeficientes correspondentes
coeficientes_features = pd.DataFrame({"Feature": X.columns, "Coeficiente": coeficientes})

# Calcular o valor absoluto dos coeficientes
coeficientes_features['Coeficiente_abs'] = coeficientes_features['Coeficiente'].apply(abs)

# Ordenar os coeficientes em ordem decrescente de valor absoluto
coeficientes_features = coeficientes_features.sort_values("Coeficiente_abs", ascending=False)

# Exibir as características com os maiores coeficientes (em valor absoluto)
coeficientes_features.head(40)


[0 1]


,Feature,Coeficiente,Coeficiente_abs
9,work_type_children,-1.149647,1.149647
1,heart_disease,1.025850,1.025850
0,hypertension,0.773980,0.773980
5,ever_married_Yes,0.533806,0.533806
6,work_type_Never_worked,-0.262053,0.262053
10,Residence_type_Urban,0.226477,0.226477
8,work_type_Self-employed,0.199972,0.199972
3,gender_Male,-0.115543,0.115543
11,smoking_status_formerly smoked,0.106283,0.106283
12,smoking_status_never smoked,-0.105708,0.105708


In [227]:
# SE DROPAR A COLUNA DE MORAR NA CIDADE OU NÃO, MUDA A ACURÁCIA DO MODELO?
X = df.drop(['stroke', 'Residence_type_Urban'], axis=1)
y = df['stroke']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model = LogisticRegression(max_iter=1000)  # Aumentar max_iter se necessário
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [228]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy * 100:.2f}%")

Acurácia do modelo: 94.26%


In [229]:
from sklearn.tree import DecisionTreeClassifier

# Treinar a árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)

# Calcular a importância das características
importances = tree.feature_importances_

# Emparelhar os nomes das características com suas importâncias
feature_importances = list(zip(X_train.columns, importances))

# Ordenar as características pela importância
sorted_feature_importances = sorted(feature_importances, key=lambda x: x[1], reverse=True)

print("Importância das características:")
feature_importance_df = pd.DataFrame(sorted_feature_importances, columns=['Feature', 'Importance'])



feature_importance_df.head(15)


Importância das características:


,Feature,Importance
0,id,0.251627
1,age,0.221166
2,avg_glucose_level,0.199811
3,bmi,0.190218
4,smoking_status_formerly smoked,0.032223
5,heart_disease,0.022679
6,hypertension,0.021535
7,work_type_Private,0.020987
8,work_type_Self-employed,0.016228
9,gender_Male,0.009269
